<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/time_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN(Recurrent Neural Network)


In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next): # dh_next는 흘러들어온 미분
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2) # 활성화 함수 tanh에 대한 미분.(1-y^2)
        db = np.sum(dt, axis=0) # repeat node에 대한 역전파.
        dWh = np.dot(h_prev.T, dt) # dot 연산에 대한 역전파
        dh_prev = np.dot(dt, Wh.T) # dot 연산에 대한 역전파
        dWx = np.dot(x.T, dt) # dot 연산에 대한 역전파
        dx = np.dot(dt, Wx.T) # dot에 연산에 대한 역전파

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

# TimeRNN(미니배치, TruncatedBPTT)

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False): # stateful은 TruncatedBPTT의 시간블록의 출력인 히든스테이트를 다음 시간블록으로 전달할지 말지 결정한다.
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 배치 크기, 시간블록 크기, embedding층을 거친 벡터 차원.
        D, H = Wx.shape # data 는 D, D를 은닉층 H 으로 보내는 Weight matirx.

        self.layers = [] # RNN 시간블록을 저장할 리스트.
        hs = np.empty((N, T, H), dtype='f') # 3차원 텐서 생성. empty는 랜덤한 값을 생성함. 히든스테이트들을 저장할 것.

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f') # 히든스테이를 다음 블록으로 전달하지 않을 경우 제로행렬을 전달.

        for t in range(T): # T는 타임사이즈.
            layer = RNN(*self.params) # 브라켓을 떼고 RNN인슨턴스 생성에 필요한 파라미터를 입력.
            self.h = layer.forward(xs[:, t, :], self.h) # 시간블록 선택(data), 히든스테이트 선택.
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs # 히든 스테이트들의 모임을 반환.

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape # 흘러들어온 미분의 shape.
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0 # dh는 다음 시간블록에서 흘러들어온 미분.
        grads = [0, 0, 0] # gradient를 저장할 리스트.
        for t in reversed(range(T)): # 역전파이므로 마지막 시간블록부터 시작.
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 흘러들어온 두 미분을 합한다.
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads): # gardient 저장.
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

# Time Embedding

In [ ]:
class TimeEmbedding:
    def __init__(self, W): # bias는 없고 W는 각 단어들의 벡터표현이 각 행에 저장돼어 있음.
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.layers = None
        self.W = W

    def forward(self, xs): # 시간 블럭으로 묶인 단어 묶음.
        N, T = xs.shape
        V, D = self.W.shape # vocab(단어 개수)사이즈 * vector의 크기

        out = np.empty((N, T, D), dtype='f') # xs와 동일한 사이즈의 행렬에 원소가 벡터로 변함.(각 단어의 ID를 벡터로 변환)
        self.layers = [] # embedding 인스턴스들을 저장

        for t in range(T): # T는 시간블럭의 길이
            layer = Embedding(self.W)
            out[:, t, :] = layer.forward(xs[:, t])
            self.layers.append(layer)

        return out

    def backward(self, dout):
        N, T, D = dout.shape # dout의 shape은 out의 shape와 동일하다.

        grad = 0
        for t in range(T):
            layer = self.layers[t]
            layer.backward(dout[:, t, :])
            grad += layer.grads[0] # 모든 gredient를 더한후 업데이트한다.

            # ? 업데이트 과정은?

        self.grads[0][...] = grad
        return None

# Time Affine

In [ ]:
class TimeAffine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x): # TimeRNN 층에서 올라온 hidden states 묶음.
        N, T, D = x.shape # 베치사이즈 * 타임사이즈 * 히든스테이트 벡터의 차원.
        W, b = self.params

        rx = x.reshape(N*T, -1) # 히든 스테이트 하나 하나 affine 변환을 하는 것 보다 행렬로 바꿔서 한번에 연산하는 것 이 빠름.
        out = np.dot(rx, W) + b # W 의 사이즈는 히든사이즈 * voc사이즈.( 각 단어의 스코어를 부여해야 하므로)
        self.x = x
        return out.reshape(N, T, -1) # 원래형태로 복원

    def backward(self, dout): # dout은 Time Softmax층에서 흘러들어온 미분.
        x = self.x
        N, T, D = x.shape
        W, b = self.params

        dout = dout.reshape(N*T, -1) # 순전파에서와의 같은 이유로 3차원텐서를 2차원 행렬로 변환.
        rx = x.reshape(N*T, -1) # rx는 TimeRNN에서 올라온 히든 스테이트 텐서를 행렬로 변환.

        db = np.sum(dout, axis=0) # repeat노드의 역전파
        dW = np.dot(rx.T, dout)
        dx = np.dot(dout, W.T)
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW # 업데이트
        self.grads[1][...] = db

        return dx # 밑으로 흘려보냄

# TimeSoftmaxWithLoss

In [ ]:
class TimeSoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], [] # S.L층은 학습해야할 파라미터가 없음.
        self.cache = None
        self.ignore_label = -1 # 특정 라벨을 무시.

    def forward(self, xs, ts): # ts는 label
        N, T, V = xs.shape # 단어에 대한 스코어.

        if ts.ndim == 3:  # 정답 레이블이 원핫 벡터인 경우
            ts = ts.argmax(axis=2) # 원핫인코딩 되기전으로 수정.

        mask = (ts != self.ignore_label)

        # 배치용과 시계열용을 정리(reshape)
        xs = xs.reshape(N * T, V) # 3차언텐서를 행렬로.
        ts = ts.reshape(N * T) # flatten
        mask = mask.reshape(N * T)

        ys = softmax(xs)
        ls = np.log(ys[np.arange(N * T), ts])# 슬라이싱 0,3  1,2  ...
        ls *= mask  # ignore_label에 해당하는 데이터는 손실을 0으로 설정
        loss = -np.sum(ls)
        loss /= mask.sum()  # corss entropy 구하는 과정

        self.cache = (ts, ys, mask, (N, T, V))
        return loss

    def backward(self, dout=1): # 마지막층이므로 흘러들어온 미분은 1
        ts, ys, mask, (N, T, V) = self.cache

        dx = ys
        dx[np.arange(N * T), ts] -= 1 # 슬라이싱, 빼기. S.L의 역전파
        dx *= dout
        dx /= mask.sum() # 개수로 나눠줌
        dx *= mask[:, np.newaxis]  # 벡터를 열벡터로하는 행렬로 변환. ignore_label에 해당하는 데이터는 기울기를 0으로 설정

        dx = dx.reshape((N, T, V))

        return dx

# Simple RNN

In [ ]:
class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size): # vocab_size = 어휘의 개수, wordvec_size = 단어를 몇차원 벡터로 변환할 것  인가?(embedding층의 출력), hidden_size = 히든 스테이트
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        embed_W = (rn(V, D) / 100).astype('f') # time embedding층의 weight matrix.
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f') # RNN층의 data에 act하는 matrix. 자비에 초기값.
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f') # RNN층의 hidden_state에 act하는 matrix. 자비에 초기값.
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f') # affine층에 act하는 matrix. 자비에 초기값.각 단어에대한 스코어 출력.
        affine_b = np.zeros(V).astype('f')

        # 계층 생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        # 모든 가중치와 기울기를 리스트에 모은다.
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params # 계속 추가.
            self.grads += layer.grads

    def forward(self, xs, ts): # xs는 단어의 묶음(배치사이즈*시간사이즈의 행렬 그 원소는 단어의 아이디). ts(배치사이즈*시간사이즈의 행렬 그 원소는 단어의 아이디)
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.rnn_layer.reset_state()

# Train Custom Loop

In [ ]:
# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100 # 단어를 100차원 벡터로 표현하고 싶음.
hidden_size = 100 # RNN의 은닉 상태 벡터의 원소 수
time_size = 5     # Truncated BPTT가 한 번에 펼치는 시간 크기
lr = 0.1
max_epoch = 100

# 학습 데이터 읽기(전체 중 1000개만)
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size] # 데이터를 1000미만으로 cut. RNN은 성능이 그렇게 좋지 못해 과거 data를 많이 기억하지 못함.
vocab_size = int(max(corpus) + 1) # 단어 id 최댓값 + 1.

xs = corpus[:-1]  # 입력. 맨끝 제외. target단어의 다음 단어가 라벨이 되므로.
ts = corpus[1:]   # 출력(정답 레이블). 맨 처음 제외.
data_size = len(xs)
print('말뭉치 크기: %d, 어휘 수: %d' % (corpus_size, vocab_size))

# 학습 시 사용하는 변수
max_iters = data_size // (batch_size * time_size) # 미니배치의 갯수. iteration 반복.
time_idx = 0 # 타임블럭
total_loss = 0
loss_count = 0
ppl_list = [] # 자연어처리에서 모델의 성능을 표현하는 식.

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

# 미니배치의 각 샘플의 읽기 시작 위치를 계산
jump = (corpus_size - 1) // batch_size
offsets = [i * jump for i in range(batch_size)] # 각 배치에서 처음등장하는 단어의 위치.

for epoch in range(max_epoch):
    for iter in range(max_iters):
        # 미니배치 취득
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets): # 각 배치 행렬에 원소를 채워넣음.
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1

        # 기울기를 구하여 매개변수 갱신
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1

    # 에폭마다 퍼플렉서티 평가
    ppl = np.exp(total_loss / loss_count)
    print('| 에폭 %d | 퍼플렉서티 %.2f'
          % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

# 그래프 그리기
x = np.arange(len(ppl_list))
plt.plot(x, ppl_list, label='train')
plt.xlabel('epochs')
plt.ylabel('perplexity')
plt.show()

# LSTM

In [ ]:
class LSTM:
    def __init__(self, Wx, Wh, b):
        '''

        Parameters
        ----------
        Wx: 입력 x에 대한 가중치 매개변수(4개분의 가중치가 담겨 있음), 옆으로 묶음.
        Wh: 은닉 상태 h에 대한 가장추 매개변수(4개분의 가중치가 담겨 있음)
        b: 편향（4개분의 편향이 담겨 있음）
        '''
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev, c_prev): # LSTM에서는 파라미터 c_prev가 하나 더 필요하다.
        Wx, Wh, b = self.params
        N, H = h_prev.shape # 히든스테이트와 shape이 똑같다.

        A = np.dot(x, Wx) + np.dot(h_prev, Wh) + b # Affine 변환.(4개의 Affine 변환)

        # 4개의 Affine변환을 sliceing
        f = A[:, :H]
        g = A[:, H:2*H]
        i = A[:, 2*H:3*H]
        o = A[:, 3*H:]

        f = sigmoid(f) # forget gate
        g = np.tanh(g) # 새로 추가할 정보.
        i = sigmoid(i) # input gate
        o = sigmoid(o) # output gate

        c_next = f * c_prev + g * i # 다음시각으로 보내는 cell state
        h_next = o * np.tanh(c_next) # 다음시각으로 보내는 hidden state

        self.cache = (x, h_prev, c_prev, i, f, g, o, c_next) # 역전파할때 사용.
        return h_next, c_next

    def backward(self, dh_next, dc_next):
        Wx, Wh, b = self.params
        x, h_prev, c_prev, i, f, g, o, c_next = self.cache

        tanh_c_next = np.tanh(c_next) # 순전파 조각.

        # 역저파 graph을 따라 연산.
        ds = dc_next + (dh_next * o) * (1 - tanh_c_next ** 2)

        dc_prev = ds * f

        di = ds * g
        df = ds * c_prev
        do = dh_next * tanh_c_next
        dg = ds * i

        di *= i * (1 - i) # sigmoid함수의 미분
        df *= f * (1 - f)
        do *= o * (1 - o)
        dg *= (1 - g ** 2) # tanh함수의 미분.

        dA = np.hstack((df, dg, di, do)) # 옆으로 이어붙임.(vstack은 vertical stcak. 위로 이어붙임.)

        dWh = np.dot(h_prev.T, dA)
        dWx = np.dot(x.T, dA)
        db = dA.sum(axis=0)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        dx = np.dot(dA, Wx.T)
        dh_prev = np.dot(dA, Wh.T)

        return dx, dh_prev, dc_prev

# TIme LSTM

In [ ]:
class TimeLSTM:
    def __init__(self, Wx, Wh, b, stateful=False): # stateful 파라미터가 추가. 순전파시 시간블럭들의 연결 여부.
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.c = None, None
        self.dh = None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        H = Wh.shape[0]

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        if not self.stateful or self.c is None:
            self.c = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = LSTM(*self.params)
            self.h, self.c = layer.forward(xs[:, t, :], self.h, self.c)
            hs[:, t, :] = self.h

            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D = Wx.shape[0]

        dxs = np.empty((N, T, D), dtype='f')
        dh, dc = 0, 0

        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh, dc = layer.backward(dhs[:, t, :] + dh, dc)
            dxs[:, t, :] = dx
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh
        return dxs

    def set_state(self, h, c=None):
        self.h, self.c = h, c

    def reset_state(self):
        self.h, self.c = None, None